In [1]:
#paquetes python
import os
import numpy as np
import copy
import pprint
from math import sqrt
from scipy.linalg import solve_triangular
import inspect
import sys

#paquetes funciones definidas obtenidas de archivos .py
import funciones_factorizacion_QR as fQR

**OBJETIVO:**

El presente notebook tiene por finalidad exhibir y poner a prueba funciones definidas por el equipo QR para resolver un Sistema de Ecuaciones Lineales (SEL), a través de 3 pasos:

    1) Obtención de factorización QR (A=QR)
    2) Implementación del método de eliminación por bloques
    3) Resolución de SEL (Ax=b) con los 2 pasos anteriores
    
Para lograr tal objetivo se presentarán a continuacion las funciones implementadas en python, así como una serie de ejemplos para entender mejor su funcionamiento

## Factorización QR

### Funciones

In [2]:
lines=inspect.getsource(fQR.crear_matriz_aleatoria)
print(lines)

def crear_matriz_aleatoria(renglones,columnas,maximo_valor,minimo_valor,entero=False):
    """
    Función de apoyo para genear matrices aleatorias

    params: renglones       no. de renglones de la matriz
            columnas        no. de columnas de la matriz
            maximo_valor    valor máximo de las entradas de la matriz
            minimo_valor    valor mínimo de las entradas de la matriz
            entero          Indica si las entradas serán enteras (True) o no
            
    return: M               Matriz con numeros al azar
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if isinstance(renglones, int)==False or isinstance(columnas, int)==False:
        sys.exit('Los parámetros de renglones y columnas deben ser enteros positivos')
    elif renglones<=0 or columnas<=0:
        sys.exit('Los parámetros de renglones y columnas deben ser enteros positivos')            
    if isinstance(maximo_valor, (int, float))==False or isinstanc

In [3]:
help(fQR.crear_matriz_aleatoria)

Help on function crear_matriz_aleatoria in module funciones_factorizacion_QR:

crear_matriz_aleatoria(renglones, columnas, maximo_valor, minimo_valor, entero=False)
    
    Descripción:
            Función que genera matrices de tamaño definido con entradas dentro del rango
            indicado
    
    Parametros:
            renglones       integer indica renglones de la matriz
            columnas        integer indica columnas de la matriz
            maximo_valor    integer o double señala valor máximo de las entradas de la matriz
            minimo_valor    integer o double señala valor mínimo de las entradas de la matriz
            entero          Boolean, indica si las entradas serán enteras (True) o no (False)
    
    Resultado:
            M               np.ndarray que representa matriz (mxn) con numeros al azar en el
                            rango indicado
    
    Ejemplos:
    Crear matriz de m=4 renglones y n=3 columnas, con entradas reales entre 3.4 y 15.6:
    cr

In [4]:
lines=inspect.getsource(fQR.house)
print(lines)

def house(x):
    """
    Función que calcula vector de householder
    
    params: x       vector al que se le hará la reflexión householder
                    
    return: Beta    factor para obtener matriz de reflexión householder Rf
            v       vector de householder
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(x) is not np.ndarray:
        sys.exit('x debe ser de tipo numpy.ndarray')
    
    #longitud del vector x=(x_0,x_1,x_2,...,x_(m-1))
    m=len(x)
    norm_2_m=x[1:m].dot(np.transpose(x[1:m]))
    #Se hace v=x=(1,x_1,x_2,...,x_(m-1))
    v=np.concatenate((1,x[1:m]), axis=None)
    Beta=0
    #con las siguientes condiciones se checa si x es múltiplo del vector canónico e_1 
    #y el signo de x[0]
    if (norm_2_m==0 and x[0]>=0):
        Beta=0
    elif (norm_2_m==0 and x[0]<0):
        Beta=2
    else:
        norm_x=np.sqrt(pow(x[0],2)+norm_2_m)
        if (x[0]<=0):
            v[0]=x[0]-norm_x
        else:
     

In [5]:
help(fQR.house)

Help on function house in module funciones_factorizacion_QR:

house(x)
    
    Descripción:
            Función que calcula vector de householder
    
    Parametros:
            x       np.ndarray que representa vector al que se le hará la reflexión householder
    
    Resultado:
            Beta    double para obtener matriz de reflexión householder v
            v       np.ndarray que representa vector que representa la reflexión de householder
    
    Con el vector v y el factor beta obtenidos se puede calcular la matriz de reflexión de
    householder R, donde R= Identidad(mxm) - Beta*v*v_transpuesta (v*v_transpuesta es matriz  
    de pxp si v tiene p entradas). El vector v está construido de tal forma que al generar la 
    matriz de reflexión de householder Rf y multiplicarse por x se obtiene una proyección de 
    este último sobre el vector canónico e_1=(1,0,0,...,0) obteniendo un vector con todas sus 
    entradas ingualas  a cero excepto la primera, que de hecho es ||x||

In [6]:
lines=inspect.getsource(fQR.matriz_auxiliar_Arv)
print(lines)

def matriz_auxiliar_Arv(A):
    """
    Función que genera una matriz que contiene los elementos r distintos de cero de la 
    matriz R y las entradas de los vectores householder v (excepto la primera), con los 
    cuales se puede calcular la matriz Q. Ambas matrices componentes de la factorización QR
    
    params: A      Matriz (mxn) de la que se desea obtner factorización QR
            
    return: Arv    Matriz (mxn) que incluye las componentes distintas de cero de la matriz R 
                   y los vectores householder con los que se puede obtener la matriz Q, y 
                   con ello la factorización QR
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray:
        sys.exit('A debe ser de tipo numpy.ndarray')
    m,n=A.shape
    if m<n:
        sys.exit('EL numero de renglones de A debe ser mayor o igual al no. de columnas')
    
    #m contiene el numero de renglones y n el de columnas
    m=A.shape[0]


In [7]:
help(fQR.matriz_auxiliar_Arv)

Help on function matriz_auxiliar_Arv in module funciones_factorizacion_QR:

matriz_auxiliar_Arv(A)
    
    Descripción:
            Función que genera una matriz que contiene los elementos r distintos de cero de 
            la matriz R y las entradas de los vectores householder v (excepto la primera), 
            con los cuales se puede calcular la matriz Q. Ambas matrices componentes de la 
            factorización QR
    
    Parametros:
            A      numpy.ndarray que representa matriz (mxn) de la que se desea obtener 
                   elemento de factorización QR
    
    Resultado:
            Arv    numpy.ndarray que representa matriz (mxn) que incluye las componentes 
                   distintas de cero de la matriz R y los vectores householder con los 
                   que se puede obtener la matriz Q, y con ello la factorización QR
    
    La factorización QR consiste en encontrar 2 matrices tales que A=Q*R 
    donde Q es una matriz ortogonal y R es una matriz 

In [8]:
lines=inspect.getsource(fQR.matriz_Q_R)
print(lines)

def matriz_Q_R(A):
    """
    Función que devuelve la matriz R y Q de la factorización QR de una matriz A
    
    params: A    Matriz (mxn)

    return: Q    Matriz Q (mxm) de la factorización A=QR
            R    Matriz Q (mxm) de la factorización A=QR
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray:
        sys.exit('A debe ser de tipo numpy.ndarray')
    elif A.shape[0]<A.shape[1]:
        sys.exit('El numero de renglones de A tiene que ser igual o mayor al no. de columnas')
    
    Arv=matriz_auxiliar_Arv(A)
    m,n=A.shape
    Q=np.eye(m)
    R=copy.copy(A)
    for j in range(n):
        Qj=Q_j(Arv,j+1)
        Q=Q@Qj
        R=Q_j(Arv,j+1)@R
    return Q,R



In [9]:
help(fQR.matriz_Q_R)

Help on function matriz_Q_R in module funciones_factorizacion_QR:

matriz_Q_R(A)
    
    Descripción:
            Función que devuelve la matriz Q y R de la factorización QR de una matriz A
    
    Parametros:
            A     np.ndarray que representa una matriz (mxn)
    
    Resultado:
            R     np.ndarray que representa matriz R (mxm) de la factorización A=QR
            Q     np.ndarray que representa matriz Q (mxm) de la factorización A=QR
    
    
    Para obtener la factorización primero se calcula la matriz auxiliar Arv. La matriz Q 
    (matriz ortogonal tal que Q_transpuesta*Q es igual a la identidad (mxm)
    se calcula como el producto de las matrices Q_j's, las cuales se obtienen a partir
    de las diferentes matrices de reflexión householder utilizadas para generar los ceros
    en las columnas de A para formar la matriz R
    
    Q = Q_1*Q_2*Q_3*Q_4
    
    Nota: Cada matriz Q_j se calcula con el vector v_j, correspondiente al j-ésimo 
    vector de house

In [10]:
lines=inspect.getsource(fQR.Q_j)
print(lines)

def Q_j(Arv,j):
    """
    Función que calcula la matriz Qj (en el proceso de obtención de factorización QR se van 
    obteniendo n Qj's, que si se multiplican todas da por resultado Q=Q_1*Q_2*...*Q_n)
                            
    params: Arv   Matriz (mxn) con la info escencial
            j     indica el índice de la matriz Q_j

    return: Qj    Matriz Q de la j-esima iteración del proceso iterativo de factorización QR
    """

    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(Arv) is not np.ndarray:
        sys.exit('Arv debe ser de tipo numpy.ndarray')
    elif Arv.shape[0]<Arv.shape[1]:
        sys.exit('El numero de renglones de Arv tiene que ser igual o mayor al no. de columnas')
    if isinstance(j, int)==False:
        sys.exit('El parámetro j deber ser un entero')
    elif 1>j or j>Arv.shape[1]:
        sys.exit('El parámetro j debe estar en el rango [1,no. columnas de Arv]')
    
    m,n=Arv.shape
    Qj=np.eye(m)
    #Para construir Q

In [11]:
help(fQR.Q_j)

Help on function Q_j in module funciones_factorizacion_QR:

Q_j(Arv, j)
    
    Descripción:
            Función que calcula la matriz Qj (en el proceso de obtención de 
            factorización QR se van obteniendo n Qj's, que si se multiplican todas 
            da por resultado Q=Q_1*Q_2*...*Q_n)
    
    Parámetros:                            
            A_r_v   np.ndarray que representa la matriz auxiliar (mxn) que incluye 
                elementos de la matriz R y Q, de la factorización QR
            j       integer que indica el índice de la matriz Q_j
    
    Resultado:
            Qj      np.ndarray que representa la j-esima transformación de householder
    
    La matriz Q_j se calcula con el vector v_j, correspondiente al j-ésimo vector 
    de householder como: Q_j = Identidad(mxm) - Beta*v_j*v_j_transpuesta



## Factorización QR

### Ejemplos

In [12]:
#Pra probar el funcionamiento de estas funciones puede definirse una matriz entrada por entrada
#o usar la función crear_matriz_aleatoria

#Ejemplo a mano (asegurandose de que sus entradas sean tipo dobles -aunque sólo se capture
#una parte entera-, pues de lo contrario se tendrán errores por redondeo significativos)
A=np.array([[1, 2, 1], [2, 3, 2], [1, 2, 3]], dtype='d')
A

array([[1., 2., 1.],
       [2., 3., 2.],
       [1., 2., 3.]])

In [13]:
x=np.array([1,2,3.])
print("x=\n",x)

Beta,v=fQR.house(x)
print("\n\nBeta=\n",np.round(Beta,4))
print("\n\nv=\n",np.round(v,4))

#Calculando matriz de reflexión de householder R
R=np.eye(3)-Beta*np.outer(v,v)
print("\n\nR=\n",np.round(R,4))

#Observese como el producto de Rx da por resultado un vector con
#todas sus entradas igualas a cero excepto la primera
print("\n\nRx=\n",np.round(R@x,4))

x=
 [1. 2. 3.]


Beta=
 0.7327


v=
 [ 1.     -0.7295 -1.0942]


R=
 [[ 0.2673  0.5345  0.8018]
 [ 0.5345  0.6101 -0.5849]
 [ 0.8018 -0.5849  0.1227]]


Rx=
 [3.7417 0.     0.    ]


In [14]:
#Definase una matriz A
A=fQR.crear_matriz_aleatoria(6,4,6,-6)
print('\n\nA=\n',np.round(A,4))

#Calculamos la matriz auxiliar Arv para visualizarla
Arv=fQR.matriz_auxiliar_Arv(A)
print('\n\nArv=\n',np.round(Arv,4))

#Obtenemos la matriz Q y R a partir de Arv
Q,R=fQR.matriz_Q_R(A)
print('\n\nQ=\n',np.round(Q,4))
print('\n\nR=\n',np.round(R,4))

#Checamos que QR=A
print('\n\nQ@R=\n',np.round(Q@R,4))
print('\n\nA=\n',np.round(A,4))



A=
 [[-2.1009 -1.8837  4.7537 -3.1834]
 [ 0.2417  3.2784 -2.193   2.485 ]
 [-3.8868  2.2063 -4.3178 -5.2122]
 [ 1.0856 -3.2768 -4.541  -0.0408]
 [-2.7726 -4.6618  4.9549 -5.4084]
 [-5.2281 -1.9616  3.9745 -1.8642]]


Arv=
 [[ 7.4685e+00  2.1152e+00 -4.4428e+00  6.9954e+00]
 [-2.5300e-02  7.1419e+00 -4.5209e+00  2.3593e+00]
 [ 4.0620e-01 -9.6620e-01  8.1757e+00  1.1798e+00]
 [-1.1340e-01  9.4100e-01  4.2940e-01  4.5067e+00]
 [ 2.8970e-01  8.8360e-01 -2.0000e-03  5.1550e-01]
 [ 5.4630e-01 -5.6300e-02  6.5800e-02 -8.4450e-01]]


Q=
 [[-0.2813 -0.1804  0.3288 -0.2613 -0.8095 -0.2379]
 [ 0.0324  0.4494 -0.0021  0.2664  0.0516 -0.8505]
 [-0.5204  0.4631 -0.5549 -0.4459 -0.0287  0.0849]
 [ 0.1454 -0.5019 -0.754   0.2254 -0.2581 -0.2028]
 [-0.3712 -0.5428  0.1042 -0.367   0.5237 -0.3844]
 [-0.7    -0.0673  0.0685  0.6902  0.0177  0.1549]]


R=
 [[ 7.4685  2.1152 -4.4428  6.9954]
 [-0.      7.1419 -4.5209  2.3593]
 [ 0.      0.      8.1757  1.1798]
 [ 0.      0.     -0.      4.5067]
 [-0.    

In [15]:
#Comparemos con el código de numpy
Q_np, R_np=np.linalg.qr(A)
#notamos que numpy obtiene con signos contrarios Q y R
print('Q_np=\n',np.round(Q_np,4))
#Además R no guarda toda la matriz R de mxn, sino sólo la parte superior de nxn
print('\nR_np=\n',np.round(R_np,4))
#la multiplicación da por resultado A
print('\nQ_np@R_np=\n',np.round(Q_np@R_np,4))

Q_np=
 [[-0.2813  0.1804  0.3288  0.2613]
 [ 0.0324 -0.4494 -0.0021 -0.2664]
 [-0.5204 -0.4631 -0.5549  0.4459]
 [ 0.1454  0.5019 -0.754  -0.2254]
 [-0.3712  0.5428  0.1042  0.367 ]
 [-0.7     0.0673  0.0685 -0.6902]]

R_np=
 [[ 7.4685  2.1152 -4.4428  6.9954]
 [ 0.     -7.1419  4.5209 -2.3593]
 [ 0.      0.      8.1757  1.1798]
 [ 0.      0.      0.     -4.5067]]

Q_np@R_np=
 [[-2.1009 -1.8837  4.7537 -3.1834]
 [ 0.2417  3.2784 -2.193   2.485 ]
 [-3.8868  2.2063 -4.3178 -5.2122]
 [ 1.0856 -3.2768 -4.541  -0.0408]
 [-2.7726 -4.6618  4.9549 -5.4084]
 [-5.2281 -1.9616  3.9745 -1.8642]]


In [16]:
#Observese que Q=(Q_4*Q_3*Q_2*Q_1)_transpuesta
np.round(np.transpose(fQR.Q_j(Arv,4)@fQR.Q_j(Arv,3)@fQR.Q_j(Arv,2)@fQR.Q_j(Arv,1)),4)

array([[-0.2813, -0.1804,  0.3288, -0.2613, -0.8095, -0.2379],
       [ 0.0324,  0.4494, -0.0021,  0.2664,  0.0516, -0.8505],
       [-0.5204,  0.4631, -0.5549, -0.4459, -0.0287,  0.0849],
       [ 0.1454, -0.5019, -0.754 ,  0.2254, -0.2581, -0.2028],
       [-0.3712, -0.5428,  0.1042, -0.367 ,  0.5237, -0.3844],
       [-0.7   , -0.0673,  0.0685,  0.6902,  0.0177,  0.1549]])

In [17]:
#Puede comprobarse que Q_transpuesta*Q es igual a la identidad
print(np.round(np.transpose(Q)@Q,4))

[[ 1. -0. -0. -0. -0. -0.]
 [-0.  1.  0.  0.  0. -0.]
 [-0.  0.  1. -0. -0. -0.]
 [-0.  0. -0.  1.  0. -0.]
 [-0.  0. -0.  0.  1. -0.]
 [-0. -0. -0. -0. -0.  1.]]


In [18]:
#Ejemplo ilustrativo de cómo obtener Q y R, visualizando cada iteración
A=fQR.crear_matriz_aleatoria(5,5,5,-5,True)
print('\n\nA=\n',np.round(A,4))

m=A.shape[0]
n=A.shape[1]
Q=np.eye(m)
R=copy.copy(A)
Arv=fQR.matriz_auxiliar_Arv(A)
for j in range(n):
    if j==0:
        cadena='Q'+str(j+1)
    else:
        cadena='Q'+str(j+1) + ' @ ' + cadena
    Qj=fQR.Q_j(Arv,j+1)
    print('\nQ',j+1,':\n',np.round(Qj,4))
    Q=Q@Qj
    R=fQR.Q_j(Arv,j+1)@R
    print('\n',cadena,'@ A=\n',np.round(R,4))

print('\n\n\nResultados finales:')
print('\nR es el resultado de multiplicar todas las Qjs a A\n',np.round(R,4))
print('\nQ es el resultado de multiplicar todas las Qjs y transponer:\n',np.round(Q,4))
print('\nQ@R:\n',np.round(Q@R,4))
print('\nA:\n',np.round(A,4))



A=
 [[ 2.  5. -3. -2.  2.]
 [ 0. -1.  5.  5.  1.]
 [ 5. -1. -3. -4. -1.]
 [ 1.  4. -3. -3. -3.]
 [ 4.  2.  5. -2.  1.]]

Q 1 :
 [[ 0.2949  0.      0.7372  0.1474  0.5898]
 [ 0.      1.      0.      0.      0.    ]
 [ 0.7372  0.      0.2292 -0.1542 -0.6166]
 [ 0.1474  0.     -0.1542  0.9692 -0.1233]
 [ 0.5898  0.     -0.6166 -0.1233  0.5067]]

 Q1 @ A=
 [[ 6.7823  2.5065 -0.5898 -5.1605  0.    ]
 [ 0.     -1.      5.      5.      1.    ]
 [-0.      1.607  -5.5199 -0.6957  1.091 ]
 [-0.      4.5214 -3.504  -2.3391 -2.5818]
 [-0.      4.0856  2.9841  0.6435  2.6728]]

Q 2 :
 [[ 1.      0.      0.      0.      0.    ]
 [ 0.     -0.1567  0.2518  0.7086  0.6403]
 [ 0.      0.2518  0.9452 -0.1543 -0.1394]
 [ 0.      0.7086 -0.1543  0.566  -0.3922]
 [ 0.      0.6403 -0.1394 -0.3922  0.6456]]

 Q2 @ Q1 @ A=
 [[ 6.7823  2.5065 -0.5898 -5.1605  0.    ]
 [-0.      6.381  -2.7459 -2.2042  0.    ]
 [-0.      0.     -3.8335  0.8728  1.3087]
 [ 0.      0.      1.2409  2.074  -1.9692]
 [-0.      0.  

In [19]:
#Se prueba un ejemplo más grande la función matriz_auxiliar_Arv
#que es base para la obtención de Q y R
A=fQR.crear_matriz_aleatoria(11**2,10**2,10,-10,False)
Arv=fQR.matriz_auxiliar_Arv(A)
Arv

array([[ 6.48428505e+01,  9.54616559e+00,  1.80734953e+00, ...,
         5.81284123e+00,  4.44555726e+00, -6.60845037e+00],
       [ 6.43909793e-02,  6.12381190e+01,  1.23759208e+01, ...,
        -1.02652634e+00,  1.00065472e+00,  6.31406307e+00],
       [ 9.55022008e-02, -1.26435010e-01,  6.00771472e+01, ...,
         2.48422428e+00, -7.91513369e-01,  4.02960943e+00],
       ...,
       [-2.03965498e-02,  1.73125719e-01, -7.31552783e-02, ...,
        -4.86030984e-01, -2.96262704e-01,  2.39843273e-01],
       [ 6.10826392e-02,  8.67458065e-02,  7.83726270e-02, ...,
         1.61102451e-02,  6.10936989e-02,  1.48438546e-02],
       [-1.05816378e-01, -5.21263302e-02, -4.19837982e-02, ...,
        -4.16923893e-01, -2.99205947e-01,  2.12789428e-01]])

## Solución de Sistemas de Ecuaciones Lineales

### Funciones

In [20]:
lines=inspect.getsource(fQR.Solucion_SEL_QR_nxn)
print(lines)

def Solucion_SEL_QR_nxn(A,b):
    """
    Función que obtiene la solución de un sistema de ecuaciones lineales (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema

    return: x   vector que satisface (Ax=b)
    """

    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray or type(b) is not np.ndarray: #esto implica que A y b tienen más de 1 elemento
        sys.exit('A y b deben ser de tipo numpy.ndarray')
    m,n = A.shape
    if m<n:
        sys.exit('EL numero de renglones de A debe ser mayor o igual al no. de columnas')
    elif b.shape[0]!= m:
    #elif b.shape[0]!= m or b.shape[1]!= 1:
        sys.exit('b debe representar un vector columna (mx1), m=no. renglones de A')
    else:
        try:
            flag=b.shape[1]
        except Exception:
            flag=1
        if flag!=1:
            sys.exit("

In [21]:
help(fQR.Solucion_SEL_QR_nxn)

Help on function Solucion_SEL_QR_nxn in module funciones_factorizacion_QR:

Solucion_SEL_QR_nxn(A, b)
    
    Descripción:
            Función que obtiene la solución de un sistema de ecuaciones lineales (SEL) con 
        n ecuaciones y n incognitas
    
    Parámetros:                            
            A      Matriz (nxn) que representa los coeficientas de las ecuaciones 
               elementos de la matriz R y Q, de la factorización QR
            b      vector (nx1) constantes del sistema
    
    Resultado:
            x      vector que satisface (Ax=b)
    
    El Sistema de Ecuaciones Lineales establecido como Ax=b, es resuelto con esta función
    mediante la factorización QR como a continuación se explica:
    
      Si A*x = b y A = Q*R, con A matriz (mxn), b (mx1) y Q una matriz ortogonal (mxm) y 
                                R (mxn) una matriz triangular superior
      entonces Q*R*x = b.
      
      Aprovechando la propiedad de Q^(-1)=Q, es decir, Q^(-1)*Q=Ide

## Eliminación por bloques

### Funciones

In [22]:
lines=inspect.getsource(fQR.crear_bloques)
print(lines)

def crear_bloques(A, b, m1=False, n1=False):
    """
    Esta es la función para la creación de bloques usando un arreglo de numpy
    
    params: A   Matriz (mxn) que representa los coeficientas de las ecuaciones
            b   vector (mx1) constantes del sistema
            n1  Numero de renglones que tendrá el 1er bloque
            m1  Numero de columnas que tendrá el 1er bloque
    
    return: A11 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            b1  Fraccion del vector dividido
            b2  Fraccion del vector dividido
    """

    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray or type(b) is not np.ndarray: #esto implica que A y b tienen más de 1 elemento
        sys.exit('A y b deben ser de tipo numpy.ndarray')
    m,n = A.shape
    if m<n:
        sys.exit('EL numero de renglones de A deb

In [23]:
help(fQR.crear_bloques)

Help on function crear_bloques in module funciones_factorizacion_QR:

crear_bloques(A, b, m1=False, n1=False)
    
    Descripción:
      Función que divide la matriz A(mxn) y el vector b(mx1) en cuatro submatrices y dos vectores respectivamente
    
    Parámetros:                            
      A     Matriz (mxn) 
      b     vector (mx1)
      m1    Número de renglones del los bloques superiores
      n1    Número de columnas de los bloques de la izquierda
    
      En el caso en que no se de m1 o n1, éstos parámetros serán igual a la parte entera del número de renglones o el número de columnas entre dos.
    
      b es un parámetro opcional, si no es introducido solo regresa las submatrices de A
    
    
    Resultado:
      A11   numpy.ndarray que representa la matriz de dimensión m1xn1
      A12   numpy.ndarray que representa la matriz de dimensión m2xn1
      A12   numpy.ndarray que representa la matriz de dimensión m1xn2
      A22   numpy.ndarray que representa la matriz 

In [24]:
lines=inspect.getsource(fQR.eliminacion_bloques)
print(lines)

def eliminacion_bloques(A,b, m1=False, n1=False):
    """
    Función que obtiene la solución de un sistema de ecuaciones lineala (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (mxn) que representa los coeficientas de las ecuaciones
            b   vector (mx1) constantes del sistema
    
    return: x1 Solucion al 1er sistema de ecuaciones obtenido con la división por bloques
            x2 Solucion al 2do sistema de ecuaciones obtenido con la división por bloques
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray or type(b) is not np.ndarray: #esto implica que A y b tienen más de 1 elemento
        sys.exit('A y b deben ser de tipo numpy.ndarray')
    m,n = A.shape
    if m<n:
        sys.exit('EL numero de renglones de A debe ser mayor o igual al no. de columnas')
        
    elif b.shape[0]!= m:
        sys.exit('b debe representar un vector columna (mx1), m=no. renglones de A')
    else:


In [25]:
help(fQR.eliminacion_bloques)

Help on function eliminacion_bloques in module funciones_factorizacion_QR:

eliminacion_bloques(A, b, m1=False, n1=False)
    
    Descripción:
      Función que usa la eliminación por bloques para resolver un sistema de ecuaciones Ax=b.
    
    Parámetros:                            
      A     numpy.ndarray que representa la matriz (mxn) que representan loc coeficientes del sistema de      ecuaciones
      b     numpy.ndarray que representa el vector (mx1) que representan el lado derecho del sistema de       ecuaciones
      m1    Número de renglones del los bloques superiores del subconjunto a resolver
      n1    Número de columnas de los bloques del subconjunto a resolver
    
      En el caso en que no se de m1 o n1, éstos parámetros serán igual a la parte entera del número de renglones o el número de columnas entre dos.
    
    
    Resultado:
      x   numpy.ndarray que representa la solución al sistema de ecuaciones Ax=b
    
    Este método consiste en eliminar un subconju

## Solución de Sistemas de Ecuaciones Lineales

### Ejemplos

### A) Sistema con soluciones únicas

In [26]:
#Generamos un sistema de ecuaciones lineales
m=12
n=12
A=np.round(fQR.crear_matriz_aleatoria(m,n,6,-6,False),2)
b = np.round(fQR.crear_matriz_aleatoria(m,1,6,-6,False),2)
print("A:",A)
print("b:",b)

A: [[-1.73 -3.47 -4.16 -5.36  0.84 -5.45 -4.98 -0.54 -5.54 -1.77  5.76  3.76]
 [-4.09  0.33  0.6   2.32 -5.85 -5.95 -1.35 -5.47  1.03  4.84  4.52  2.77]
 [-1.2  -3.28  2.49  1.53  2.49 -4.99 -5.45 -3.82 -0.34 -2.2   5.91 -2.36]
 [ 1.78  2.03  4.69 -3.48 -5.7  -4.18  2.19  3.95 -2.88 -1.28 -5.22 -3.99]
 [ 2.52 -2.01 -4.1   0.09 -0.97 -4.63  1.44  3.07 -1.44 -1.86  0.4  -1.66]
 [ 3.03 -5.96 -5.28 -2.72 -1.29  3.47 -2.91 -0.13  0.38 -2.44 -2.3   2.39]
 [ 3.74 -1.47 -3.27 -1.26 -4.22 -5.82  1.63 -0.48  3.27  3.03  5.89 -2.83]
 [ 0.77 -2.38  2.28  2.06 -3.89  4.69  4.12  4.57 -2.01 -2.44 -5.42  5.92]
 [-4.38 -4.27 -0.61  1.27 -5.22 -2.35  0.96 -5.74 -4.75  3.61  5.88  2.15]
 [-5.   -3.6  -2.94 -1.03 -1.39 -3.16 -4.28  3.44 -3.35 -3.34  1.37 -5.5 ]
 [ 0.63 -5.02  2.77 -5.09  2.58 -4.43 -0.23 -1.14  2.63  5.86  4.28  1.51]
 [-5.52  2.86 -1.   -4.47  1.03  0.15 -3.12 -2.09 -0.5  -5.36 -3.71  2.81]]
b: [[ 4.89]
 [ 2.01]
 [-2.4 ]
 [ 1.24]
 [-1.86]
 [-4.35]
 [-4.84]
 [ 3.81]
 [ 3.32]
 [-2.48]
 [-

Primero resolvamos el sistema de ecuaciones usando la paquetería de numpy para comparar.

In [27]:
np.linalg.solve(A,b)

array([[ 5.52334977e-02],
       [ 5.81121801e-01],
       [ 2.34978880e-01],
       [ 1.16914994e-01],
       [ 2.63362950e-04],
       [ 6.23594098e-02],
       [-2.32115472e-01],
       [ 3.53451975e-01],
       [-9.35628452e-01],
       [ 4.02030917e-01],
       [ 2.07118604e-01],
       [ 6.15957349e-01]])

Ahora usemos la factorización QR

In [28]:
x=fQR.Solucion_SEL_QR_nxn(A,b)
x

array([[ 5.52334977e-02],
       [ 5.81121801e-01],
       [ 2.34978880e-01],
       [ 1.16914994e-01],
       [ 2.63362950e-04],
       [ 6.23594098e-02],
       [-2.32115472e-01],
       [ 3.53451975e-01],
       [-9.35628452e-01],
       [ 4.02030917e-01],
       [ 2.07118604e-01],
       [ 6.15957349e-01]])

Por último usemos eliminación por bloques con QR

In [29]:
fQR.eliminacion_bloques(A,b)

array([[ 5.52334977e-02],
       [ 5.81121801e-01],
       [ 2.34978880e-01],
       [ 1.16914994e-01],
       [ 2.63362950e-04],
       [ 6.23594098e-02],
       [-2.32115472e-01],
       [ 3.53451975e-01],
       [-9.35628452e-01],
       [ 4.02030917e-01],
       [ 2.07118604e-01],
       [ 6.15957349e-01]])

### B) Sistema sin solución

In [30]:
# #Generamos un sistema de ecuaciones lineales
# A = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 0]], dtype='d')
# b = np.array([[3], [3], [5]], dtype='d')
# print("A:")
# pprint.pprint(A)
# print("b:")
# pprint.pprint(b)

Primero resolvamos el sistema de ecuaciones usando la paquetería de numpy para comparar.

In [31]:
# np.linalg.solve(A,b)

* Marca un error: "Singular matrix"

Ahora usemos la factorización QR

In [32]:
# x=fQR.Solucion_SEL_QR_nxn(A,b)
# x

In [33]:
#Checamos que sea solucion (debe ser igual al vector b)
# A@x

Por último usemos eliminación por bloques con QR

In [34]:
# fQR.eliminacion_bloques(A,b)

* Se concluye que es un sistema de rectas paralelas, por lo tanto no existe solúción.

### C) Sistema con infinitas soluciones

In [35]:
# #Generamos un sistema de ecuaciones lineales
# A = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 0]], dtype='d')
# b = np.array([[3], [3], [0]], dtype='d')
# print("A:")
# pprint.pprint(A)
# print("b:")
# pprint.pprint(b)

Primero resolvamos el sistema de ecuaciones usando la paquetería de numpy para comparar.

In [36]:
# np.linalg.solve(A,b)

* Marca un error: "Singular matrix"

Ahora usemos la factorización QR

In [37]:
# x=fQR.Solucion_SEL_QR_nxn(A,b)
# x

In [38]:
#Checamos que sea solucion (debe ser igual al vector b)
# A@x

Por último usemos eliminación por bloques con QR

In [39]:
# fQR.eliminacion_bloques(A,b)